In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.figure as fgr
from matplotlib.pyplot import figure
import seaborn as sns
import time
import os

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler,StandardScaler, RobustScaler, MaxAbsScaler, LabelEncoder, OneHotEncoder, PowerTransformer
from sklearn.metrics import mean_squared_error,accuracy_score,recall_score,precision_score,f1_score,roc_auc_score
from sklearn.model_selection import train_test_split, RepeatedKFold, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import make_scorer, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score
import pylab
from scipy.stats import skew

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [3]:
train_df_normal = pd.read_csv("Normal_data.csv")
train_df_metasploitable = pd.read_csv("metasploitable-2.csv")
train_df_OVS = pd.read_csv("OVS.csv")

In [4]:
print(train_df_normal.shape)
print(train_df_normal.info())

print(train_df_metasploitable.shape)
print(train_df_metasploitable.info())

print(train_df_OVS.shape)
print(train_df_OVS.info())

(68424, 84)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68424 entries, 0 to 68423
Data columns (total 84 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow ID            68424 non-null  object 
 1   Src IP             68424 non-null  object 
 2   Src Port           68424 non-null  int64  
 3   Dst IP             68424 non-null  object 
 4   Dst Port           68424 non-null  int64  
 5   Protocol           68424 non-null  int64  
 6   Timestamp          68424 non-null  object 
 7   Flow Duration      68424 non-null  int64  
 8   Tot Fwd Pkts       68424 non-null  int64  
 9   Tot Bwd Pkts       68424 non-null  int64  
 10  TotLen Fwd Pkts    68424 non-null  float64
 11  TotLen Bwd Pkts    68424 non-null  float64
 12  Fwd Pkt Len Max    68424 non-null  int64  
 13  Fwd Pkt Len Min    68424 non-null  int64  
 14  Fwd Pkt Len Mean   68424 non-null  float64
 15  Fwd Pkt Len Std    68424 non-null  float64
 16  Bwd Pkt Le

In [5]:
train_df_normal.head().T

,0,1,2,3,4
Flow ID,185.127.17.56-192.168.20.133-443-53648-6,185.127.17.56-192.168.20.133-443-53650-6,192.168.20.133-192.168.20.2-35108-53-6,192.168.20.133-192.168.20.2-35108-53-6,154.59.122.74-192.168.20.133-443-60900-6
Src IP,185.127.17.56,192.168.20.133,192.168.20.133,192.168.20.2,192.168.20.133
Src Port,443,53650,35108,53,60900
Dst IP,192.168.20.133,185.127.17.56,192.168.20.2,192.168.20.133,154.59.122.74
Dst Port,53648,443,53,35108,443
...,...,...,...,...,...
Idle Mean,0.0,0.0,0.0,0.0,0.0
Idle Std,0.0,0.0,0.0,0.0,0.0
Idle Max,0.0,0.0,0.0,0.0,0.0
Idle Min,0.0,0.0,0.0,0.0,0.0


In [6]:
train_df_metasploitable.head().T

,0,1,2,3,4
Flow ID,192.168.3.130-200.175.2.130-38694-4444-6,192.168.3.130-200.175.2.130-38693-4444-6,192.168.3.130-200.175.2.130-3632-33747-6,192.168.3.130-200.175.2.130-8180-38745-6,192.168.3.130-200.175.2.130-8180-37217-6
Src IP,192.168.3.130,192.168.3.130,200.175.2.130,200.175.2.130,200.175.2.130
Src Port,38694,38693,33747,38745,37217
Dst IP,200.175.2.130,200.175.2.130,192.168.3.130,192.168.3.130,192.168.3.130
Dst Port,4444,4444,3632,8180,8180
...,...,...,...,...,...
Idle Mean,0.0,0.0,0.0,0.0,0.0
Idle Std,0.0,0.0,0.0,0.0,0.0
Idle Max,0.0,0.0,0.0,0.0,0.0
Idle Min,0.0,0.0,0.0,0.0,0.0


In [7]:
train_df_OVS.head().T

,0,1,2,3,4
Flow ID,172.17.0.2-200.175.2.130-80-48856-6,172.17.0.2-200.175.2.130-80-48856-6,172.17.0.2-200.175.2.130-80-48858-6,172.17.0.2-200.175.2.130-80-48858-6,172.17.0.2-200.175.2.130-80-48860-6
Src IP,200.175.2.130,200.175.2.130,200.175.2.130,200.175.2.130,200.175.2.130
Src Port,48856,48856,48858,48858,48860
Dst IP,172.17.0.2,172.17.0.2,172.17.0.2,172.17.0.2,172.17.0.2
Dst Port,80,80,80,80,80
...,...,...,...,...,...
Idle Mean,0.0,0.0,0.0,0.0,0.0
Idle Std,0.0,0.0,0.0,0.0,0.0
Idle Max,0.0,0.0,0.0,0.0,0.0
Idle Min,0.0,0.0,0.0,0.0,0.0


In [8]:
print(train_df_normal.Label.value_counts())
print(train_df_metasploitable.Label.value_counts())
print(train_df_OVS.Label.value_counts())


Label
Normal    68424
Name: count, dtype: int64
Label
DDoS     73529
Probe    61757
DoS       1145
BFA        295
U2R         17
Name: count, dtype: int64
Label
DoS           52471
DDoS          48413
Probe         36372
BFA            1110
Web-Attack      192
BOTNET          164
Name: count, dtype: int64


In [9]:
print(train_df_normal.isnull().sum())


Flow ID      0
Src IP       0
Src Port     0
Dst IP       0
Dst Port     0
            ..
Idle Mean    0
Idle Std     0
Idle Max     0
Idle Min     0
Label        0
Length: 84, dtype: int64


In [10]:
print(train_df_metasploitable.isnull().sum())

Flow ID      0
Src IP       0
Src Port     0
Dst IP       0
Dst Port     0
            ..
Idle Mean    0
Idle Std     0
Idle Max     0
Idle Min     0
Label        0
Length: 84, dtype: int64


In [11]:
print(train_df_OVS.isnull().sum())

Flow ID      0
Src IP       0
Src Port     0
Dst IP       0
Dst Port     0
            ..
Idle Mean    0
Idle Std     0
Idle Max     0
Idle Min     0
Label        0
Length: 84, dtype: int64


In [12]:
train_df_metasploitable.describe()

,Src Port,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,136743.000000,136743.000000,136743.000000,1.367430e+05,136743.000000,136743.000000,136743.000000,136743.000000,136743.000000,136743.000000,...,136743.000000,136743.0,1.367430e+05,1.367430e+05,1.367430e+05,1.367430e+05,1.367430e+05,1.367430e+05,1.367430e+05,1.367430e+05
mean,7584.451285,11313.557747,2.796172,8.394408e+06,0.926951,3.091632,8.757662,8.173025,8.036229,0.001638,...,0.298231,0.0,1.511978e+04,4.731904e+03,2.083635e+04,1.228448e+04,8.143571e+06,1.375585e+04,8.156480e+06,8.132080e+06
std,15980.548414,17724.142526,2.994221,2.138543e+07,1.744029,1.982729,22.970945,15.063613,13.315169,0.235716,...,0.812670,0.0,2.242634e+05,1.100321e+05,2.819971e+05,2.124402e+05,2.099977e+07,3.157609e+05,2.100513e+07,2.099902e+07
min,0.000000,0.000000,0.000000,1.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000,0.000000,0.000000,1.600000e+01,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000,0.000000,0.000000,3.300000e+01,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,80.000000,33915.500000,6.000000,1.021000e+04,3.000000,4.000000,30.000000,30.000000,30.000000,0.000000,...,1.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,65518.000000,58178.000000,17.000000,1.199654e+08,84.000000,87.000000,2460.000000,2498.000000,287.000000,44.000000,...,82.000000,0.0,4.575740e+06,3.366629e+06,4.807377e+06,4.575740e+06,1.200000e+08,9.274678e+06,1.200000e+08,1.200000e+08


In [13]:
train_df_OVS.describe()

,Src Port,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,138722.000000,138722.000000,138722.000000,1.387220e+05,138722.000000,138722.000000,138722.000000,138722.000000,138722.000000,138722.000000,...,138722.000000,138722.0,1.387220e+05,1.387220e+05,1.387220e+05,1.387220e+05,1.387220e+05,1.387220e+05,1.387220e+05,1.387220e+05
mean,28590.150805,4058.269092,3.911492,1.705513e+06,8.533751,2.950650,149.048759,292.504022,68.814644,0.026211,...,1.360224,0.0,5.027155e+04,1.583963e+03,5.242607e+04,4.941142e+04,1.420137e+06,4.633044e+04,1.468712e+06,1.380748e+06
std,23855.062703,11594.755381,2.877929,1.107296e+07,2445.718229,14.918974,1935.634892,741.152741,194.641622,0.962083,...,47.433114,0.0,7.341994e+05,7.711422e+04,7.476735e+05,7.325348e+05,1.024988e+07,7.443023e+05,1.036689e+07,1.020332e+07
min,0.000000,0.000000,0.000000,1.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000,0.000000,0.000000,2.000000e+00,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,37831.000000,80.000000,6.000000,2.208000e+03,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,48394.000000,1058.000000,6.000000,7.661750e+03,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,63020.000000,65389.000000,17.000000,1.197861e+08,910748.000000,2039.000000,77600.000000,80143.000000,1448.000000,44.000000,...,2425.000000,0.0,6.810000e+07,6.092367e+06,6.810000e+07,6.810000e+07,8.220000e+07,1.780000e+07,8.220000e+07,8.220000e+07


In [14]:
numeric_df = train_df_metasploitable.select_dtypes(include=['int64', 'float64'])
object_df = train_df_metasploitable.select_dtypes(include=['object'])
print('Числовые: ', numeric_df.columns)
print('Объектные: ', object_df.columns)

Числовые:  Index(['Src Port', 'Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Av

In [ ]:
import pandas as pd

train_df_normal['Total Packets'] = train_df_normal['Tot Fwd Pkts'] + train_df_normal['Tot Bwd Pkts']
average_packets_per_flow = train_df_normal['Total Packets'].mean()

print(f"train_df_normal: Среднее количество пакетов в потоке: {average_packets_per_flow}")

train_df_metasploitable['Total Packets'] = train_df_metasploitable['Tot Fwd Pkts'] + train_df_metasploitable['Tot Bwd Pkts']
average_packets_per_flow = train_df_metasploitable['Total Packets'].mean()

print(f"train_df_metasploitable: Среднее количество пакетов в потоке: {average_packets_per_flow}")

train_df_OVS['Total Packets'] = train_df_OVS['Tot Fwd Pkts'] + train_df_OVS['Tot Bwd Pkts']
average_packets_per_flow = train_df_OVS['Total Packets'].mean()

print(f"train_df_OVS: Среднее количество пакетов в потоке: {average_packets_per_flow}")

train_df_normal: Среднее количество пакетов в потоке: 30.3998450835964
train_df_metasploitable: Среднее количество пакетов в потоке: 4.018582304030188
train_df_OVS: Среднее количество пакетов в потоке: 11.484400455587433


In [ ]:
import pandas as pd

flow_counts_per_host = train_df_normal.groupby('Src IP').size()

average_flows = flow_counts_per_host.mean()
min_flows = flow_counts_per_host.min()
max_flows = flow_counts_per_host.max()
print("train_df_normal")
print(f"Среднее количество потоков с хоста: {average_flows}")
print(f"Минимальное количество потоков с хоста: {min_flows}")
print(f"Максимальное количество потоков с хоста: {max_flows}")

flow_counts_per_host = train_df_metasploitable.groupby('Src IP').size()

average_flows = flow_counts_per_host.mean()
min_flows = flow_counts_per_host.min()
max_flows = flow_counts_per_host.max()
print("train_df_metasploitable")
print(f"Среднее количество потоков с хоста: {average_flows}")
print(f"Минимальное количество потоков с хоста: {min_flows}")
print(f"Максимальное количество потоков с хоста: {max_flows}")

flow_counts_per_host = train_df_OVS.groupby('Src IP').size()

average_flows = flow_counts_per_host.mean()
min_flows = flow_counts_per_host.min()
max_flows = flow_counts_per_host.max()
print("train_df_OVS")
print(f"Среднее количество потоков с хоста: {average_flows}")
print(f"Минимальное количество потоков с хоста: {min_flows}")
print(f"Максимальное количество потоков с хоста: {max_flows}")


train_df_normal
Среднее количество потоков с хоста: 74.21258134490239
Минимальное количество потоков с хоста: 1
Максимальное количество потоков с хоста: 53790
train_df_metasploitable
Среднее количество потоков с хоста: 1.859664631243965
Минимальное количество потоков с хоста: 1
Максимальное количество потоков с хоста: 37362
train_df_OVS
Среднее количество потоков с хоста: 2.864973151590252
Минимальное количество потоков с хоста: 1
Максимальное количество потоков с хоста: 84947


In [ ]:
import pandas as pd


all_ips = pd.concat([train_df_normal['Src IP'], train_df_normal['Dst IP']]).unique()
num_hosts_approx = len(all_ips)
print("train_df_normal")
print(f"Приблизительное количество хостов: {num_hosts_approx}")
num_flows = len(train_df_normal)
print(f"Количество потоков: {num_flows}")

all_ips = pd.concat([train_df_metasploitable['Src IP'], train_df_metasploitable['Dst IP']]).unique()
num_hosts_approx = len(all_ips)
print("train_df_metasploitable")
print(f"Приблизительное количество хостов: {num_hosts_approx}")
num_flows = len(train_df_metasploitable)
print(f"Количество потоков: {num_flows}")

all_ips = pd.concat([train_df_OVS['Src IP'], train_df_OVS['Dst IP']]).unique()
num_hosts_approx = len(all_ips)
print("train_df_OVS")
print(f"Приблизительное количество хостов: {num_hosts_approx}")
num_flows = len(train_df_OVS)
print(f"Количество потоков: {num_flows}")

train_df_normal
Приблизительное количество хостов: 1086
Количество потоков: 68424
train_df_metasploitable
Приблизительное количество хостов: 73532
Количество потоков: 136743
train_df_OVS
Приблизительное количество хостов: 48423
Количество потоков: 138722


In [21]:
print(train_df_metasploitable["Label"])

0         U2R
1         U2R
2         U2R
3         BFA
4         BFA
         ... 
136738    U2R
136739    U2R
136740    U2R
136741    U2R
136742    U2R
Name: Label, Length: 136743, dtype: object
